In [114]:
import os
import glob
import math
import pandas as pd
import numpy as np

from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import clima_anom as ca
import cartopy
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%run ../function/function.ipynb

In [1]:
# so = 'manjaro'
so = 'xubuntu'

In [2]:
if so == 'xubuntu':
    base = '/media/'
elif so == 'manjaro':
    base = '/run/media/'
print(f'so  : {so}')
print(f'base: {base}')

so  : xubuntu
base: /media/


In [115]:
shape = list(shpreader.Reader('../mappings/SaoPaulo_Fixed.shp').geometries())

In [116]:
Brasildat_dir = base + 'arturo/Arturo/Data/BrasilDat/SP/BrasilDat_2018_03_5min_8kmx8km.nc'
Brasildat = ca.read_netcdf(Brasildat_dir,2)
lat = Brasildat['lat']
lon = Brasildat['lon']
time_B = Brasildat['time']
flash_B = Brasildat['flash']

lon2d, lat2d = np.meshgrid(lon,lat)
nt, ny, nx = np.shape(flash_B)

lat : (45,)
lon : (45,)
time : (8928,)
flash : (8928, 45, 45)


In [117]:
GLM_dir = base + 'arturo/Arturo/Data/GLM/SP/GLM_2018_03_5min_8kmx8km.nc'
GLM = ca.read_netcdf(GLM_dir,2)
flash_G = GLM['flash']

lat : (45,)
lon : (45,)
time : (8928,)
flash : (8928, 45, 45)


## Cleaning data

In [118]:
lat_c = -22.8611
lon_c = -47.0563
r = (180)/111

In [119]:
Area_R = np.zeros([ny,nx])
for i in range(ny):
    for j in range(nx):
        Dist = distancia(lat_c,lon_c,lat[i],lon[j])
        if Dist <= 162:
            Area_R[i,j] = 1

In [120]:
G_new = np.zeros([nt,ny,nx])
B_new = np.zeros([nt,ny,nx])
for t in range(nt):
    for i in range(ny):
        for j in range(nx):
            if Area_R[i,j] == 1:
                G_new[t,i,j] = flash_G[t,i,j]
                B_new[t,i,j] = flash_B[t,i,j]
            else:
                G_new[t,i,j] = np.nan
                B_new[t,i,j] = np.nan

## 15 mintues accumulated

In [124]:
ntime = int(nt/3)
print(ntime)
flash_B_15 = np.zeros([ntime, ny, nx])
flash_G_15 = np.zeros([ntime, ny, nx])
for t in range(ntime):
    # print(t,3*t,3*t+3)
    flash_B_15[t,:,:] = np.sum(B_new[3*t:3*t+3,:,:],axis=0)
    flash_G_15[t,:,:] = np.sum(G_new[3*t:3*t+3,:,:],axis=0)

2976


## BrasilDat Export netcdf

In [127]:
print(f'Brasildat shape: {flash_B_15.shape}')
info = {'file': base + 'arturo/Arturo/Datos/BrasilDat/SP/BrasilDat_2018_03_15min_8kmx8km.nc', # filename and directory
        'title': 'Brasildat data 8km x 8km', # netcdf title
        'year_start':2018,'month_start':3,'day_start':1,'hour_start':0,'minute_start':0, # time start (year,month,day,hour,minute)
        'year_end':2018,'month_end':3,'day_end':31,'hour_end':23,'minute_end':59, # time end (year,month,day,hour,minute)
        'time_frequency': 'Minutely', # options: Monthly, Daily, Hourly, Minutely
        'time_interval':15,
        'var_name': 'flash', # variable short name
        'var_units': 'flash/196km2'} # variable units

Brasildat shape: (2976, 45, 45)


In [128]:
ca.create_netcdf(info,flash_B_15,lat,lon)

Time Start: 2018-03-01 00:00:00
Time End: 2018-03-31 23:45:00
Time Frequency: Minutely
Time Lenght: 2976

File created in:  /media/arturo/Arturo/Datos/BrasilDat/SP/BrasilDat_2018_03_15min_8kmx8km.nc
File title:  Brasildat data 8km x 8km
var title:  flash
var units:  flash/196km2


## GLM Export netcdf

In [129]:
print(f'Brasildat shape: {flash_G_15.shape}')
info = {'file': base + 'arturo/Arturo/Datos/GLM/SP/GLM_2018_03_15min_8kmx8km.nc', # filename and directory
        'title': 'GLM-flash data 8km x 8km', # netcdf title
        'year_start':2018,'month_start':3,'day_start':1,'hour_start':0,'minute_start':0, # time start (year,month,day,hour,minute)
        'year_end':2018,'month_end':3,'day_end':31,'hour_end':23,'minute_end':59, # time end (year,month,day,hour,minute)
        'time_frequency': 'Minutely', # options: Monthly, Daily, Hourly, Minutely
        'time_interval':15,
        'var_name': 'flash', # variable short name
        'var_units': 'flash/196km2'} # variable units

Brasildat shape: (2976, 45, 45)


In [130]:
ca.create_netcdf(info,flash_G_15,lat,lon)

Time Start: 2018-03-01 00:00:00
Time End: 2018-03-31 23:45:00
Time Frequency: Minutely
Time Lenght: 2976

File created in:  /media/arturo/Arturo/Datos/GLM/SP/GLM_2018_03_15min_8kmx8km.nc
File title:  GLM-flash data 8km x 8km
var title:  flash
var units:  flash/196km2
